In [1]:
# Load data ke dalam data frame 
import pandas as pd

# Spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('scraping_azarine-neg.csv', encoding='latin-1')

df.head()

,Review,Product,Rating,Username,Date
0,"Pengiriman sangat lama, 3hari setelah order ba...",C-White Serum,bintang 1,Nocturnal,Lebih dari 1 tahun lalu
1,Salah kirim barang,C White Night Moisturizer,bintang 1,N***i,Lebih dari 1 tahun lalu
2,Paket Acne nya tidak lengkap Facial foamnya ti...,Paket Intense Acne & repair Series,bintang 1,Faridah,Lebih dari 1 tahun lalu
3,Packing tidak baik. Packing luar rusak & sdh t...,C-White Serum,bintang 1,Yanti,Lebih dari 1 tahun lalu
4,barang tak pernah sampai kaga amanh lama bange...,AZARINE EASY WHITE PAKET HEMAT,bintang 1,S***a,Lebih dari 1 tahun lalu


In [2]:
# Drop kolom yang tidak terpakai
df = df.drop(df.iloc[:,1:], axis=1)

# Cek data teratas
df.head()

,Review
0,"Pengiriman sangat lama, 3hari setelah order ba..."
1,Salah kirim barang
2,Paket Acne nya tidak lengkap Facial foamnya ti...
3,Packing tidak baik. Packing luar rusak & sdh t...
4,barang tak pernah sampai kaga amanh lama bange...


PREPROCESSING

In [3]:
# PROSES CASE FOLDING (menjadikan semua huruf di kolom Review menjadi huruf kecil)
df['Review'] = df['Review'].str.lower()

print('Case Folding Result: \n')
df.head()

Case Folding Result: 



,Review
0,"pengiriman sangat lama, 3hari setelah order ba..."
1,salah kirim barang
2,paket acne nya tidak lengkap facial foamnya ti...
3,packing tidak baik. packing luar rusak & sdh t...
4,barang tak pernah sampai kaga amanh lama bange...


In [4]:
# import hasil case folding menjadi csv
df.to_csv("casefolding_azarine.csv")

In [5]:
# CLEANING
import re # regex library

def remove_tweet_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t'," ").replace('\\n', " ").replace('\\u', " ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Review'] = df['Review'].apply(remove_tweet_special)

# remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Review'] = df['Review'].apply(remove_number)

# remove punctuation and replace with space
def remove_punctuation(text):
    return re.sub(r'[.,]', ' ', text)

df['Review'] = df['Review'].apply(remove_punctuation)

# remove punctuation
def remove_symbol(text):
    # Menghapus simbol-simbol tidak standar dan menggantinya dengan spasi
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Menghapus multiple whitespace
    cleaned_text = re.sub('\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

df['Review'] = df['Review'].apply(remove_symbol)

# Menentukan ambang batas panjang string acak
threshold_length = 20

# Fungsi untuk menghapus string acak berdasarkan panjangnya
def hapus_string_acak_dengan_panjang(text):
    # Memeriksa panjang string dan menghapus jika melebihi ambang batas
    return ' '.join(word for word in text.split() if len(word) <= threshold_length)

# Menggunakan fungsi untuk menghapus string acak pada kolom 'Text'
df['Review'] = df['Review'].apply(hapus_string_acak_dengan_panjang)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Review'] = df['Review'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Review'] = df['Review'].apply(remove_whitespace_multiple)

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Review'] = df['Review'].apply(remove_single_char)

def remove_laughter(text):
    laughter_patterns = r'\b((ha)+h*|(he)+h*|(hi)+h*|(wk)+w*k*|(eh)+e*|(ah)+a*|(ih)+i*|(kw)+k*w*|(hem)+m*)\b'
    return re.sub(laughter_patterns, '', text, flags=re.IGNORECASE)

df['Review'] = df['Review'].apply(remove_laughter)

print('Cleaning Result : \n') 
print(df.head())

Cleaning Result : 

                                              Review
0  pengiriman sangat lama hari setelah order baru...
1                                 salah kirim barang
2  paket acne nya tidak lengkap facial foamnya ti...
3  packing tidak baik packing luar rusak sdh terb...
4  barang tak pernah sampai kaga amanh lama bange...


In [6]:
# import hasil cleaning menjadi csv
df.to_csv("cleaning_azarine.csv")

In [7]:
# REMOVE DUPLICATE
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# Menghapus baris yang kosong
df = df.dropna(subset=['Review'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['Review'].str.strip() != '']
df.head()

,Review
0,pengiriman sangat lama hari setelah order baru...
1,salah kirim barang
2,paket acne nya tidak lengkap facial foamnya ti...
3,packing tidak baik packing luar rusak sdh terb...
4,barang tak pernah sampai kaga amanh lama bange...


In [8]:
# import hasil remove duplicate menjadi csv
df.to_csv("removedup_azarine.csv")

In [9]:
# NORMALIZATION
import pandas as pd
import re

slang_dictionary = pd.read_csv('../colloquial-indonesian-lexicon2.csv')
slang_dict = pd.Series(slang_dictionary['formal'].values,index=slang_dictionary['slang']).to_dict()

slang_dictionary.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1.0,wow,elongasi,0,0
1,aminn,amin,1.0,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1.0,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1.0,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0.0,Birthday yg keberpa kak?,abreviasi,0,0


In [10]:
# Normalisasi kata menggunakan kamus colloquial-indonesian-lexicon2.csv
def Slangwords(text, slang_dict):
    for word in text.split():
        if word in slang_dict.keys():
            # menambahkan \b untuk menandakan batas kata di sekitar kata slang
            text = re.sub(r'\b{}\b'.format(re.escape(word)), slang_dict[word], text)
    
    text = re.sub('@[\w]+', '', text)
    return text

df['Review_norm'] = df['Review'].apply(lambda x: Slangwords(x, slang_dict))
print(df.head())

                                              Review  \
0  pengiriman sangat lama hari setelah order baru...   
1                                 salah kirim barang   
2  paket acne nya tidak lengkap facial foamnya ti...   
3  packing tidak baik packing luar rusak sdh terb...   
4  barang tak pernah sampai kaga amanh lama bange...   

                                         Review_norm  
0  pengiriman sangat lama hari setelah order baru...  
1                                 salah kirim barang  
2  paket acne nya tidak lengkap facial foamnya ti...  
3  packing tidak baik packing luar rusak sudah te...  
4  barang tak pernah sampai kagak amanh lama bang...  


In [11]:
# import hasil replace slang words menjadi csv
df.to_csv("normalization_azarine.csv")

In [12]:
# PROSES TOKENIZING (word_tokenize() untuk memecah string kedalam tokens)

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Tokenizing
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['review_tokens'] = df['Review_norm'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df.head())

Tokenizing Result : 

                                              Review  \
0  pengiriman sangat lama hari setelah order baru...   
1                                 salah kirim barang   
2  paket acne nya tidak lengkap facial foamnya ti...   
3  packing tidak baik packing luar rusak sdh terb...   
4  barang tak pernah sampai kaga amanh lama bange...   

                                         Review_norm  \
0  pengiriman sangat lama hari setelah order baru...   
1                                 salah kirim barang   
2  paket acne nya tidak lengkap facial foamnya ti...   
3  packing tidak baik packing luar rusak sudah te...   
4  barang tak pernah sampai kagak amanh lama bang...   

                                       review_tokens  
0  [pengiriman, sangat, lama, hari, setelah, orde...  
1                             [salah, kirim, barang]  
2  [paket, acne, nya, tidak, lengkap, facial, foa...  
3  [packing, tidak, baik, packing, luar, rusak, s...  
4  [barang, tak, pernah, samp

In [13]:
# import hasil tokenizing menjadi csv
df.to_csv("tokenizing_azarine.csv")

In [14]:
# PROSES FILTERING (Stopword Removal)
from nltk.corpus import stopwords

# get stopword indonesia from NLTK stopword
list_stopwords = stopwords.words('indonesian')

# convert list to dictionary
list_stopwords = set(list_stopwords)

# remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['review_tokens_SR'] = df['review_tokens'].apply(stopwords_removal) 

print(df.head())

                                              Review  \
0  pengiriman sangat lama hari setelah order baru...   
1                                 salah kirim barang   
2  paket acne nya tidak lengkap facial foamnya ti...   
3  packing tidak baik packing luar rusak sdh terb...   
4  barang tak pernah sampai kaga amanh lama bange...   

                                         Review_norm  \
0  pengiriman sangat lama hari setelah order baru...   
1                                 salah kirim barang   
2  paket acne nya tidak lengkap facial foamnya ti...   
3  packing tidak baik packing luar rusak sudah te...   
4  barang tak pernah sampai kagak amanh lama bang...   

                                       review_tokens  \
0  [pengiriman, sangat, lama, hari, setelah, orde...   
1                             [salah, kirim, barang]   
2  [paket, acne, nya, tidak, lengkap, facial, foa...   
3  [packing, tidak, baik, packing, luar, rusak, s...   
4  [barang, tak, pernah, sampai, kagak, amanh,

In [15]:
# import hasil stopword removal menjadi csv
df.to_csv("stopword_azarine.csv")

In [16]:
# PROSES LEMMATIZATION 
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['review_tokens_SR']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['review_tokens_stemmed'] = df['review_tokens_SR'].swifter.apply(get_stemmed_term)
print(df['review_tokens_stemmed'])
print(df.head())

1265
------------------------
pengiriman : kirim
order : order
dikirim : kirim
barang : barang
nya : nya
salah : salah
kirim : kirim
paket : paket
acne : acne
lengkap : lengkap
facial : facial
foamnya : foamnya
packing : packing
rusak : rusak
terbuka : buka
terima : terima
diterima : terima
orang : orang
rumah : rumah
serum : serum
penyok : penyok
jual : jual
kagak : kagak
amanh : amanh
banget : banget
bakl : bakl
belanja : belanja
tanggal : tanggal
transaksi : transaksi
syarat : syarat
member : member
platinum : platinum
gagal : gagal
gara : gara
semoga : moga
kecewa : kecewa
buang : buang
tong : tong
sampah : sampah
product : product
pelindungnya : lindung
menandakan : tanda
segel : segel
plastik : plastik
maaf : maaf
kasih : kasih
bintang : bintang
diperhatikan : perhati
ya : ya
perihal : perihal
kemasan : kemas
dikonfirmasi : konfirmasi
pesan : pesan
mei : mei
konfirmasi : konfirmasi
produk : produk
sesuai : sesuai
urutan : urut
orderan : order
pengajuan : aju
pick : pick
up : up
j

Pandas Apply: 100%|██████████| 347/347 [00:00<00:00, 115675.05it/s]

0                     [kirim, order, kirim, barang, nya]
1                                 [salah, kirim, barang]
2           [paket, acne, nya, lengkap, facial, foamnya]
3      [packing, packing, rusak, buka, terima, terima...
4                   [barang, kagak, amanh, banget, bakl]
                             ...                        
343    [barang, terima, order, produk, item, kirim, s...
344    [order, tanggal, mei, pas, libur, kirim, tau, ...
345    [segel, barang, pas, buka, kayak, habis, pakai...
346                            [order, dipickup, barang]
347    [barang, terima, tanggal, september, status, t...
Name: review_tokens_stemmed, Length: 347, dtype: object
                                              Review  \
0  pengiriman sangat lama hari setelah order baru...   
1                                 salah kirim barang   
2  paket acne nya tidak lengkap facial foamnya ti...   
3  packing tidak baik packing luar rusak sdh terb...   
4  barang tak pernah sampai kaga aman

In [17]:
# import hasil preprocessing menjadi csv
df.to_csv("lemma_azarine.csv")